In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgokmen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complained object qwoqqr does not exist in the database. Skipping.
Complained object dnypbe does not exist in the database. Skipping.
Complained object upmkjr does not exist in the database. Skipping.
Complained object wmchsj does not exist in the database. Skipping.
Complained object uqusgj does not exist in the database. Skipping.
Complained object urmqxk does not exist in the database. Skipping.
Complained object aginyz does not exist in the database. Skipping.
Complained object pcpvvu does not exist in the database. Skipping.
Complained object fveyao does not exist in the database. Skipping.
Complained object knunaz does not exist in the database. Skipping.
Complained object didlbp does not exist in the database. Skipping.
Complained object didlbp does not exist in th

In [2]:
# Get the substance synsets that are missing a system
for s in Synset.all_objects():
  if s.is_substance and len(s.matching_particle_systems) == 0:
    print(s)

In [3]:
# Get the particle systems that are missing a particle
missing_particle = ParticleSystem.view_error_missing_particle()
print(len(missing_particle))

456


In [4]:
# Remove the particle systems that are cooked
missing_particle_not_cooked = [ps for ps in missing_particle if "cooked__" not in ps.name]
print(len(missing_particle_not_cooked))
print(missing_particle_not_cooked)

191
[ParticleSystem(name='clove', parameters='{"type": "granular", "particle_density": 1050.0}'), ParticleSystem(name='coconut', parameters='{"type": "macro_physical_particle", "particle_density": 1000.0}'), ParticleSystem(name='diced__agave', parameters='{"type": "macro_physical_particle", "particle_density": 400.0}'), ParticleSystem(name='diced__apricot', parameters='{"type": "macro_physical_particle", "particle_density": 655.0}'), ParticleSystem(name='diced__arepa', parameters='{"type": "macro_physical_particle", "particle_density": 501.0}'), ParticleSystem(name='diced__artichoke', parameters='{"type": "macro_physical_particle", "particle_density": 634.0}'), ParticleSystem(name='diced__arugula', parameters='{"type": "macro_physical_particle", "particle_density": 180.0}'), ParticleSystem(name='diced__asparagus', parameters='{"type": "macro_physical_particle", "particle_density": 566.0}'), ParticleSystem(name='diced__auricularia', parameters='{"type": "macro_physical_particle", "parti

In [5]:
# Remove the particle systems that are not diced
missing_particle_not_cooked_not_diced = [ps for ps in missing_particle_not_cooked if "diced__" not in ps.name]
print(len(missing_particle_not_cooked_not_diced))
print(missing_particle_not_cooked_not_diced)

5
[ParticleSystem(name='clove', parameters='{"type": "granular", "particle_density": 1050.0}'), ParticleSystem(name='coconut', parameters='{"type": "macro_physical_particle", "particle_density": 1000.0}'), ParticleSystem(name='granola', parameters='{"type": "macro_physical_particle", "particle_density": 467.0}'), ParticleSystem(name='kidney_bean', parameters='{"type": "macro_physical_particle", "particle_density": 748.0}'), ParticleSystem(name='orzo', parameters='{"type": "macro_physical_particle", "particle_density": 710.0}')]


In [6]:
# Count by group
import collections
ctr = collections.Counter([("cooked__" in ps.name, "diced__" in ps.name) for ps in missing_particle])
print("Uncooked Diced:", ctr[(False, True)])
print("Cooked Diced:", ctr[(True, True)])
print("Uncooked Non-Diced:", ctr[(False, False)])
print("Cooked Non-Diced:", ctr[(True, False)])

Uncooked Diced: 186
Cooked Diced: 195
Uncooked Non-Diced: 5
Cooked Non-Diced: 70


In [7]:
# How many non-liquid particle systems are there anyway?
print("Total non-liquid particle systems:", sum([1 for s in Synset.all_objects() if s.is_substance and not s.is_liquid]))

Total non-liquid particle systems: 670


In [11]:
# And how many diced?
print("Total diced particle systems:", sum([1 for s in Synset.all_objects() if s.is_substance and "diced__" in s.name]))
print("Diced particle systems w/ particles:", sum([1 for s in Synset.all_objects() if s.is_substance and "diced__" in s.name and s.state == SynsetState.MATCHED]))

Total diced particle systems: 405
Diced particle systems w/ particles: 24


In [9]:
# Get a list of all cookable substances
for s in Synset.all_objects():
  if s.is_substance and "cookable" in s.property_names:
    print(s)

Synset(name='allspice.n.03')
Synset(name='almond.n.02')
Synset(name='barbecue_sauce.n.01')
Synset(name='beaten_egg.n.01')
Synset(name='beef_broth.n.01')
Synset(name='beef_stew.n.01')
Synset(name='beer.n.01')
Synset(name='black_bean.n.01')
Synset(name='black_pepper.n.02')
Synset(name='blueberry.n.02')
Synset(name='blueberry_compote.n.01')
Synset(name='blueberry_mousse.n.01')
Synset(name='breadcrumb.n.01')
Synset(name='brown_rice.n.01')
Synset(name='brown_sugar.n.01')
Synset(name='brownie_batter.n.01')
Synset(name='cafe_au_lait.n.01')
Synset(name='cake_mix.n.01')
Synset(name='cashew.n.02')
Synset(name='catsup.n.01')
Synset(name='cayenne.n.02')
Synset(name='cheese_pastry_filling.n.01')
Synset(name='cheese_sauce.n.01')
Synset(name='cherry_filling.n.01')
Synset(name='chia_seed.n.01')
Synset(name='chicken_broth.n.01')
Synset(name='chicken_curry.n.01')
Synset(name='chicken_soup.n.01')
Synset(name='chickpea.n.03')
Synset(name='chocolate_sauce.n.01')
Synset(name='cinnamon.n.03')
Synset(name='ci

In [13]:
# For all existing particle systems, get the average max extent of their particles
import numpy as np
systems_and_sizes = []
for ps in ParticleSystem.all_objects():
  if not any("macro" in pn for pn in ps.synset.property_names):
    continue
  if len(ps.particles) > 0:
    avg = np.mean([np.max(p.bounding_box_size) for p in ps.particles])
    systems_and_sizes.append((ps, avg))

# Sort by size
systems_and_sizes.sort(key=lambda x: x[1])
for ps, size in systems_and_sizes:
  print(ps, size)

grated_cheese 0.003
green_tea 0.003
diced__chili 0.005
diced__clove 0.005
diced__lettuce 0.005
diced__spinach 0.005
diced__green_onion 0.009395132700602214
chickpea 0.009440608978271484
diced__apple 0.01
diced__avocado 0.01
diced__beefsteak_tomato 0.01
diced__beet 0.01
diced__bell_pepper 0.01
diced__carrot 0.01
diced__cheddar 0.01
diced__chicken 0.01
diced__cucumber 0.01
diced__feta 0.01
diced__kiwi 0.01
diced__lemon 0.01
diced__mango 0.01
diced__pineapple 0.01
diced__potato 0.01
diced__vidalia_onion 0.01
diced__virginia_ham 0.01
diced__watermelon 0.01
diced__zucchini 0.01
black_bean 0.010039367735385895
soy 0.01026789649327596
mulch 0.010625
chocolate_kiss 0.010672941207885742
pea 0.01072930246591568
blueberry 0.011368206818898519
cat_food 0.011420027256011963
jelly_bean 0.012087002158164979
pine_nut 0.014425103425979614
pill 0.014469870448112487
cranberry 0.01498478349049886
pellet_food 0.015
raisin 0.015034597941807337
ravioli 0.015920995712280273
coffee_bean 0.017965644836425782
no

In [22]:
# Diced-not-cooked particle system record for Wensi's pass
import json
diced_not_cooked = []
for ps in missing_particle_not_cooked:
  if "diced__" in ps.name and "cooked__" not in ps.name:
    # Get the definition of the derivative parent
    parents = [da for da in ps.synset.derivative_ancestors if not da.is_derivative and "sliced__" not in da.name]
    definition = None
    if len(parents) > 0:
        assert len(parents) == 1, f"Expected 1 parent for {ps.name}, got {parents}"
        parent = parents[0]
        definition = parent.definition
    diced_not_cooked.append((ps.name, definition))
with open("diced_particle_systems.json", "w") as f:
  json.dump(diced_not_cooked, f, indent=4)